CLASIFICACIÓN DE IMÁGENES CON LENGUA DE SEÑAS.
Aguilar Guzmán Naomi Betel
416089231 \\
BD: https://kaggle.com/datamunge/sign-language-mnist

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import keras
import time

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing import image
import os
import glob
from keras.applications.resnet import ResNet50
from keras.applications.vgg19 import VGG19
from keras.utils import plot_model
from keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1
import shutil

Primero accedemos al drive donde esta la base de datos y las imagenes de muestra:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = 'gdrive/Mi unidad/Base de datos/'

La base de datos se encuentra en formato csv. La columna 'label' encontramos las etiquetas de las imagenes, la etiqueta es un numero del  al 25, a manera de: 0 es A, 1 es B y etc. El resto de las columnas contienen un valor del 0 al 255 que representa el color de los pixeles, por lo que hay 28*28=784 columnas.


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Base de datos/sign_mnist_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Base de datos/sign_mnist_test.csv')
train = train.sample(frac=1)
test = test.sample(frac=1)
train

In [ ]:
test

Definimos una lista para las etiquetas de las letras como string, para facilitar la visualizacion de los datos:

In [ ]:
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M','N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y','Z']

Ahora pasamos la base de datos a un formao de numpy:

In [ ]:
train_data = np.array(train, dtype = 'float32')
test_data = np.array(test, dtype = 'float32')

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
print('TIPOS DE CLASE')
print('Imprime la clase de train:', type(train))
print('Imprime la clase de test:', type(test))

print('TAMAÑO')
print('Dimensiones de train', train.shape)
print('Dimensiones de test', test.shape)

print('PORCENTAJES')
total=test.shape[0]+train.shape[0]
print('Tenemos ', total, 'imágenes en total.')
print('El porcentaje de imágenes de entrenamiento es: ', train.shape[0]/total)
print('El porcentaje de imágenes de prueba es: ', test.shape[0]/total)

Visualizamos una muestra aleatoria de los elementos de la base de datos. Como se observa, es una imagen de 28x28 pixeles en escala de grises con su respectiva etiqueta:

In [ ]:
i = random.randint(1,train.shape[0])
fig1, ax1 = plt.subplots(figsize=(2,2))
plt.imshow(train_data[i,1:].reshape((28,28)), cmap='gray')
print("Esta letras es: ", class_names[int(train_data[i,0])])

Ahora hacemos un histograma para ver la distribucion de las muestras. Como se observa, las muestras son en general balanceadas.

In [ ]:

for i in range(0,25):
  train['label'].replace({i:class_names[i]},inplace=True)
fig = plt.figure(figsize=(18,18))
ax1 = fig.add_subplot(221)
train['label'].value_counts().plot(kind='bar', ax=ax1)
ax1.set_ylabel('Train')
ax1.set_xlabel('Letra')
ax1.set_title('Histograma de la base de datos')

for i in range(0,25):
  test['label'].replace({i:class_names[i]},inplace=True)
fig = plt.figure(figsize=(18,18))
ax1 = fig.add_subplot(221)
test['label'].value_counts().plot(kind='bar', ax=ax1)
ax1.set_ylabel('Test')
ax1.set_xlabel('Letra')
ax1.set_title('Histograma de la base de datos')

Ahora separamos normalizamos la base de datos y usamos reshape para ponerla en formato (27455,28,28,1) pues hay 27455 imagenes de 28*28 pixeles y en escala de grises.

In [ ]:
x_train = train_data[:, 1:] /255
x_test = test_data[:, 1:] /255
x_train = x_train.reshape(x_train.shape[0], 28 ,28 ,1)
x_test = x_test.reshape(x_test.shape[0], 28, 28 , 1)
x_train.shape

In [ ]:
y_train = train_data[:, 0]
y_test = test_data[:,0]
y_train

Ahora preparamos la estructura de una red convolutiva:

In [ ]:
model1 = Sequential()

model1.add(Conv2D(32, (3, 3), input_shape = (28,28,1), activation='relu'))
model1.add(MaxPooling2D(pool_size = (2, 2)))
model1.add(Dropout(0.3))

model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size = (2, 2)))
model1.add(Dropout(0.3))

model1.add(Conv2D(128, (3, 3), activation='relu',bias_regularizer=l1(0.05)))
model1.add(MaxPooling2D(pool_size = (2, 2)))
model1.add(Dropout(0.3))

model1.add(Conv2D(128, (1, 1), activation='relu',bias_regularizer=l1(0.05)))
model1.add(MaxPooling2D(pool_size = (1, 1)))
model1.add(Dropout(0.3))


model1.add(Flatten())

model1.add(Dense(128, activation = 'sigmoid'))
model1.add(Dropout(0.3))
model1.add(Dense(25, activation = 'softmax'))

Esquema del modelo implementado

In [ ]:
plot_model(model1, to_file = 'mnist_signal.png', show_shapes = True, rankdir = 'TB', show_layer_names = True)

Ahora compilamos el modelo y mostramos sus parámetros:

> Bloque con sangría



In [ ]:

model1.compile(loss ='sparse_categorical_crossentropy', optimizer='adam', metrics =['accuracy'])
model1.summary()

Entrenando el modelo:

In [ ]:
history = model1.fit(x_train, y_train, batch_size = 128, epochs = 20, verbose = 1, validation_data = (x_test, y_test))

Ahora graficamos train loss y el validation loss para verificar que no haya sobreenrenamiento:

In [ ]:
#GRAFICAMOS
plt.figure(figsize = (8,6))
plt.plot(history.history['loss'], color = 'green')
plt.plot(history.history['val_loss'], color = 'red')
plt.ylabel('Cost', size = 16)
plt.xlabel('Epoch', size = 16)
plt.legend(['Train', 'Validation'], loc = 'upper right', fontsize = 16)
plt.show()

In [ ]:
#GRAFICAMOS
plt.figure(figsize = (8,6))
plt.plot(history.history['accuracy'], color = 'green')
plt.plot(history.history['val_accuracy'], color = 'blue')
plt.ylabel('Accuracy', size = 16)
plt.xlabel('Epoch', size = 16)
plt.legend(['Train', 'Validation'], loc = 'lower right', fontsize = 16)
plt.show()

In [ ]:
#EVALUACION
evaluations = model1.evaluate(x = x_test[:], y = y_test[:])
print ("Loss:" + str(evaluations[0]))
print ("Test Accuracy:" + str(evaluations[1]))

Ahora hacemos una prediccion sobre los datos de prueba, para verificar que letras son las que son peor clasificadas y graficamos un hisograma con el porcenaje de muestras mal clasificadas:

In [ ]:
prediction = model1.predict(x_test)
classes_x=np.argmax(prediction,axis=1)

count1=[0 for i in class_names]
i=0
while i<len(y_test):
  count1[int(y_test[i])]+=1
  i+=1

count2=[0 for i in class_names]
i=0
while i<len(classes_x):
  if int(classes_x[i]) != int(y_test[i]):
    count2[int(y_test[i])]+=1
  i+=1

comp=[]
i=0
while i<len(class_names):
  if count1[i] !=0:
    comp.append([i,(count2[i]/count1[i])*100])
  else:
    comp.append([i,0])
  i+=1

comp.sort(key=lambda x: -x[1])
class_label=[class_names[int(comp[i][0])] for i in range(0,len(class_names))]

comp=pd.DataFrame(comp)
for i in range(0,25):
  comp[0].replace({i:class_names[i]},inplace=True)

fig = plt.figure(figsize=(18,18))
ax1 = fig.add_subplot(221)
comp.plot(kind='bar', ax=ax1)
ax1.set_xticklabels(class_label)
ax1.set_ylabel('Clasificaciones incorrectas (%)')
ax1.set_xlabel('Letra')
ax1.set_title('Histograma de letras mal clasificadas')
plt.show()

Ahora tomamos una muestra aleatoria de la base de datos y checamos que prediccion da el modelo:

In [ ]:
i = random.randint(0,len(prediction)-1)
plt.imshow(x_test[i,:,:,0])
print("Letra Predecida: ", class_names[int(classes_x[i])])
print("Letra Verdadera: ", class_names[int(y_test[i])])

In [ ]:
counter=0

while True:
  i = random.randint(0,len(prediction)-1)
  counter+=1

  if class_names[int(y_test[i])]!=class_names[int(classes_x[i])]:
    print("Después de ", counter, 'intentos hubo una predicción errónea.\n')
    plt.imshow(x_test[i,:,:,0])
    print("Letra Predecida: ", class_names[int(classes_x[i])])
    print("Letra Verdadera: ", class_names[int(y_test[i])])
    break